In [8]:
import os
import pandas as pd
import io
import unicodedata

root_dir = 'coral_and_sponges_all/data/pub/data/paleo/coral'


In [11]:
root_files = os.listdir(root_dir)
root_files = root_files[0:3] + root_files[4:6] + root_files[9:12]
sub_files_to_flatten = [[f'{r}/{f}' for f in os.listdir(f'{root_dir}/{r}')] for r in root_files]
#Flatten list of files into one list
sub_files = [f'{root_dir}/{item}' for sublist in sub_files_to_flatten for item in sublist]

In [12]:
len(sub_files)

139

In [131]:
def create_coral_file_df(filename):
    try:
        # Using a weird encoding to solve file read issue; see https://stackoverflow.com/questions/46000191/utf-8-codec-cant-decode-byte-0x92-in-position-18-invalid-start-byte
        coral_file = open(filename,'r', encoding="cp1252")
        file_in = coral_file.readlines()

        for i in range(len(file_in)):
            line = file_in[i]
            #print(line)
            if "Location:" in line:
                location = line.split(": ")[1].strip()
            if "Northernmost_Latitude:" in line:
                northernmost_lat = line.split(": ")[1].strip()
            if "Southernmost_Latitude:" in line:
                southernmost_lat = line.split(": ")[1].strip()
            if "Easternmost_Longitude:" in line:
                easternmost_lat = line.split(": ")[1].strip()
            if "Westernmost_Longitude:" in line:
                westernmost_lat = line.split(": ")[1].strip()
            if "Elevation:" in line:
                elevation = line.split(": ")[1].split(' ')[0].strip()
            if "Earliest_Year:" in line:
                earliest_year = line.split(": ")[1].strip()
            if "Most_Recent_Year:" in line:
                most_recent_year = line.split(": ")[1].strip()
            if "Species_Name:" in line:
                species = line.split(": ")[1].strip('\n').strip()
            if not line.startswith('#'):
                if line.startswith('ï»¿#'):
                    continue #this is the first line with special characters hidden
                elif len(line.strip()) == 0:
                    continue
                else:
                    age_line = i
                    extracted = ' '.join(file_in[age_line:])
                    break #exit the for loop, we have obtained the data!

        lat = (float(northernmost_lat) + float(southernmost_lat)) / 2
        lon = (float(easternmost_lat) + float(westernmost_lat)) / 2

        data = io.StringIO(extracted)
        df = pd.read_csv(data, sep="\t", error_bad_lines=False)

        df.columns = (map(lambda x: x.lower(), df.columns))

        # find the correct 'age' column
        if 'age' not in df.columns:
            if 'age_ad' in df.columns:
                df['age'] = df['age_ad']
            elif 'time' in df.columns:
                df['age'] = df['time']
            elif 'date' in df.columns:
                df['age'] = df['date']
            elif 'year' in df.columns:
                df['age'] = df['year']
            elif 'year_CE' in df.columns:
                df['age'] = df['year_CE']
            else:
                print("No 'age' column header detected.")

        #print(df['age'])

        #find the correct d18O column
        if 'd18o' not in df.columns:
            count = 0
            for col in df.columns:
                # we are assuming that the first col with d18O is the one we want
                if 'd18o' in col:
                    df['d18o'] = df[col]
                    break
        
        #print(df['d18o'])

        df = df[['age', 'd18o']]
        df = df[df['d18o'] != 'NAN']

        # average the d18O values across the years
        df['year']  = df['age'].apply(lambda x: str(x).strip()[0:4])
        df = df.groupby(by="year")["d18o"].mean().reset_index('year')

        df['lat'] = lat
        df['lon'] = lon
        df['elevation'] = elevation
        
        try:
            species
            df['species'] = species
        except NameError:
            df['species'] = 'Other'

        return(df)

        # except Exception as e:
        #     print("exception2")
        #     print("ERROR : "+str(e))
        #     # print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))
            # print('hi')
    except Exception as e:
        print("ERROR : "+str(e))

Quick Proof of Concept for the function above:

In [132]:
#filename = f'{root_dir}/atlantic/barbados_coral18o.txt'
# filename = f'{root_dir}/atlantic/goodkin2008-sst_noaa.txt'
filename = f'{root_dir}/atlantic/smith2006-lk1.txt'
#'swart1998-st-1_noaa.txt'

create_coral_file_df(filename)

/var/folders/ws/d7dc4bz53q3_26hs2vbmg8600000gn/T/ipykernel_15832/844505233.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  create_coral_file_df(filename)


,year,d18o,lat,lon,elevation,species
0,1947,-4.058571,24.556667,-81.4125,-4,Other
1,1948,-4.191429,24.556667,-81.4125,-4,Other
2,1949,-4.144000,24.556667,-81.4125,-4,Other
3,1950,-3.943846,24.556667,-81.4125,-4,Other
4,1951,-3.905333,24.556667,-81.4125,-4,Other
5,1952,-3.922143,24.556667,-81.4125,-4,Other
6,1953,-4.170000,24.556667,-81.4125,-4,Other
7,1954,-4.013125,24.556667,-81.4125,-4,Other
8,1955,-3.916111,24.556667,-81.4125,-4,Other
9,1956,-3.869000,24.556667,-81.4125,-4,Other


In [133]:
all_coral_df = pd.DataFrame(columns=['age', 'd18O', 'lat', 'lon', 'elevation', 'species'])
for file in sub_files:
    print(file)
    coral_df = create_coral_file_df(file)
    all_coral_df = pd.concat([all_coral_df, coral_df])
del all_coral_df['age']
del all_coral_df['d18O']
all_coral_df.reset_index
all_coral_df


coral_and_sponges_all/data/pub/data/paleo/coral/atlantic/moses2006-side_noaa.txt


UnboundLocalError: local variable 'northernmost_lat' referenced before assignment

In [109]:
all_coral_df.to_csv('all_coral.csv')


In [108]:
all_coral_df

,lat,lon,elevation,species,year,d18o
0,16.77,-22.88,-3,Siderastrea radians,1928,-3.110000
1,16.77,-22.88,-3,Siderastrea radians,1929,-3.008333
2,16.77,-22.88,-3,Siderastrea radians,1930,-2.835000
3,16.77,-22.88,-3,Siderastrea radians,1931,-2.855833
4,16.77,-22.88,-3,Siderastrea radians,1932,-2.936667
...,...,...,...,...,...,...
150,1.00,173.00,-6,,1990,-4.923150
151,1.00,173.00,-6,,1991,-5.083814
152,1.00,173.00,-6,,1992,-4.850820
153,1.00,173.00,-6,,1993,-4.899430


In [ ]:
coral_file = 'swart1998-st-1_noaa.txt'
# Example of file with -noaa extension
file1 = open(coral_file,'r')
print(file1.read())
file1.close()

Gather metadata info! (site lat/lon & years of study)

In [ ]:
file1 = open(coral_file,'r')
file_in = file1.readlines()

for line in file_in:
    if "Location:" in line:
        location = line.split(": ")[1]
    if "Northernmost_Latitude:" in line:
        northernmost_lat = line.split(": ")[1]
    if "Southernmost_Latitude:" in line:
        southernmost_lat = line.split(": ")[1]
    if "Easternmost_Longitude:" in line:
        easternmost_lat = line.split(": ")[1]
    if "Westernmost_Longitude:" in line:
        westernmost_lat = line.split(": ")[1]
    if "Elevation:" in line:
        elevation = line.split(": ")[1].split(' ')[0]
    if "Earliest_Year:" in line:
        earliest_year = line.split(": ")[1]
    if "Most_Recent_Year:" in line:
        most_recent_year = line.split(": ")[1]
    if "Species_Name:" in line:
        species = line.split(": ")[1].strip('\n')

lat = (float(northernmost_lat) + float(southernmost_lat)) / 2
lon = (float(easternmost_lat) + float(westernmost_lat)) / 2



In [ ]:
age_line = 0
file1 = open(coral_file,'r')
with file1 as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('age'):
            print(i)
            age_line = i
            break
    extracted = ' '.join(lines[age_line:])
    print(extracted)

In [ ]:
data = io.StringIO(extracted)
df_test = pd.read_csv(data, sep="\t")
df_test.head()

Just grab `age` and `d18O` columns! And remove NAN values. 

In [ ]:
df_test_d18O = df_test[['age', 'd18O']]
df_test_d18O = df_test_d18O[df_test_d18O['d18O'] != 'NAN']
df_test_d18O['lat'] = lat
df_test_d18O['lon'] = lon
df_test_d18O['elevation'] = elevation
df_test_d18O['species'] = species
df_test_d18O